首先看一下我们需要达成的效果:
Pattern: (我想要A)
Response: (如果你有 A，对你意味着什么呢？)

Input: (我想要度假)
Response: (如果你有度假，对你意味着什么呢？)

为了实现模板的判断和定义，我们需要定义一个特殊的符号类型，这个符号类型就叫做"variable"， 这个"variable"用来表示是一个占位符。例如，定义一个目标: "I want X"， 我们可以表示成 "I want ?X", 意思就是?X是一个用来占位的符号。

如果输入了"I want holiday"， 在这里 'holiday' 就是 '?X'
所以我们目标完成的函数如下:

In [78]:
def get_response(saying, rules):
    """" please implement the code, to get the response as followings:
    
    >>> get_response('I need iPhone') 
    >>> Image you will get iPhone soon
    >>> get_response("My mother told me something")
    >>> Talk about more about your monther.
    """
    pass

那我们先编写一个判断是否是variable的函数，逻辑很简单传入参数第一位如果是X，并且之后所有位是字母，则返回True

In [1]:
def is_variable(pat):
    return pat.startswith('?') and all(s.isalpha() for s in pat[1:])

接下来完成匹配函数，函数对传入的两个列表进行匹配，返回当中所有variable的值。这里的思路是两个列表第一位开始不断递归调用匹配函数，边界为匹配不上的情况或者传入为空的情况(可能是匹配结束了)，此时返回空列表。

In [5]:
def pat_match(pattern, saying):
    #若传入的列表存在一个空值则匹配结束
    if not pattern or not saying: 
        return []
    #判断当前位是否为variable
    if is_variable(pattern[0]):
        #返回variable对应的值，递归匹配下一位。最终返回的是所有[(pattern[0], saying[0])]和达到递归边界时的最后一个空列表的和。
        return [(pattern[0], saying[0])] + pat_match(pattern[1:], saying[1:])
    else:
        #匹配不上时匹配结束
        if pattern[0] != saying[0]: 
            return []
        else:
            #递归调用继续匹配下一位
            return pat_match(pattern[1:], saying[1:])

In [6]:
pat_match("?X greater than ?Y".split(), "3 greater than 2".split())

[('?X', '3'), ('?Y', '2')]

In [7]:
pat_match("?X greater than ?Y".split(), "3 greater ".split())

[('?X', '3')]

此时我们的模式是逐字逐句匹配的， "I need iPhone" 和 "I need ?X" 可以匹配，但是"I need an iPhone" 和 "I need ?X" 就不匹配了，那怎么办？

为了解决这个问题，我们可以新建一个变量类型 "?\*X", 这种类型多了一个星号(\*),表示匹配多个。自然也有了判断是否是新变量的函数:

In [8]:
def is_pattern_segment(pattern):
    return pattern.startswith('?*') and all(a.isalpha() for a in pattern[2:]) #注意[2:]是因为?*有两位，占了字符串的0，1位

在正式进入我们的方法之前，先写几个工具性质的函数。

下面这个函数递归比较输入列表的每一位，若不相同返回False，若全部相同或者rest中存在变量返回True

In [39]:
def is_match(rest, saying):
    if not rest and not saying:           #注意这里是and，双方均为空列表时才全部匹配
        return True
    if not rest or not saying:  #两个列表没有同时为空，一个为空说明两列表长度不匹配。这里不直接比较输入的长度是因为输入可能包含变量
        return False    
    if not all(a.isalpha() for a in rest[0]):  #包含变量视为无条件匹配
        return True
    if rest[0] != saying[0]:
        return False
    return is_match(rest[1:], saying[1:])

In [40]:
is_match('very good'.split(), 'very good'.split())

True

In [41]:
is_match('very good'.split(), 'not very good'.split())

False

In [42]:
is_match('?*P is very good'.split(), 'not very good'.split()) #因为包含变量?*P，所以返回True

True

In [43]:
is_match('very good'.split(), 'very good and cool'.split())

False

下面这个函数的输入pattern是以变量为第一个元素的列表，作用是完成pattern和saying的最大匹配。比如

输入:

segment_match('?*P is very good'.split(), "My dog and my cat is very good".split())

输出:

(('?P', ['My', 'dog', 'and', 'my', 'cat']), 5)   

In [44]:
def segment_match(pattern, saying):
    seg_pat, rest = pattern[0], pattern[1:]   
    seg_pat = seg_pat.replace('?*', '?')     #seg_pat==['?P']，rest==['is','very','good']
                                             #saying==['My','dog','and','my','cat','is','very','good']
    if not rest:
        return (seg_pat, saying), len(saying)    
    #寻找rest[1:]与saying的公共连续子串，匹配到后返回子串前的内容
    for i, token in enumerate(saying):
        if rest[0] == token and is_match(rest[1:], saying[(i + 1):]): #rest[0]==['is']，rest[1:]==['very','good']
            return (seg_pat, saying[:i]), i
    #若匹配不上，则将整个saying返回
    return (seg_pat, saying), len(saying)

In [45]:
segment_match('?*P is very good'.split(), "My dog and my cat is very good".split())

(('?P', ['My', 'dog', 'and', 'my', 'cat']), 5)

In [46]:
segment_match('?*P is very good'.split(), "My dog and my cat is cool".split()) #若匹配不上，则将整个saying返回

(('?P', ['My', 'dog', 'and', 'my', 'cat', 'is', 'cool']), 7)

In [47]:
segment_match('?*P is very good'.split(), "My dog and my cat is very good and cool".split())

(('?P',
  ['My', 'dog', 'and', 'my', 'cat', 'is', 'very', 'good', 'and', 'cool']),
 10)

In [48]:
segment_match("?X greater than ?Y".split(), "3 greater than 2".split())#可以看到对于存在多个变量，segment_match目前只能匹配第一个

(('?X', ['3']), 1)

现在我们对pat_match函数改写，使其能匹配"?\*X"。先将初版的pat_match贴过来

In [51]:
def pat_match(pattern, saying):
    #若传入的列表存在一个空值则匹配结束
    if not pattern or not saying: 
        return []
    #判断当前位是否为variable
    if is_variable(pattern[0]):
        #返回variable对应的值，递归匹配下一位。最终返回的是所有[(pattern[0], saying[0])]和达到递归边界时的最后一个空列表的和。
        return [(pattern[0], saying[0])] + pat_match(pattern[1:], saying[1:])
    else:
        #匹配不上时匹配结束
        if pattern[0] != saying[0]: 
            return []
        else:
            #递归调用继续匹配下一位
            return pat_match(pattern[1:], saying[1:])

In [141]:
def pat_match_with_seg(pattern, saying):
    if not pattern or not saying: 
        return []
   #这里因为 pattern[0]会多次用到所以先提出来
    pat = pattern[0]
    #这一步和之前一样
    if is_variable(pat):
        return [(pat, saying[0])] + pat_match_with_seg(pattern[1:], saying[1:])
    #引入新的函数，判断变量是否为?*X。是的话则用segment_match完成最大匹配
    elif is_pattern_segment(pat):
        match, index = segment_match(pattern, saying)
        return [match] + pat_match_with_seg(pattern[1:], saying[index:])
    #递归调用继续匹配下一位
    elif pat == saying[0]:
        return pat_match_with_seg(pattern[1:], saying[1:])
    else:
        return '匹配失败'

In [142]:
pat_match_with_seg("?X greater than ?Y".split(), "3 greater than 2".split())

[('?X', '3'), ('?Y', '2')]

In [143]:
pat_match_with_seg('?*P is very good and ?*X'.split(), "My dog is very good and my cat is very cute".split())

[('?P', ['My', 'dog']), ('?X', ['my', 'cat', 'is', 'very', 'cute'])]

In [144]:
pat_match_with_seg('is very good and'.split(), "is very cute".split())

'匹配失败'

In [145]:
pat_match_with_seg('is very good and ?*X'.split(), "is very good and my cat is very cute".split())

[('?X', ['my', 'cat', 'is', 'very', 'cute'])]

为了方便接下来的替换工作，我们新建立两个函数，一个是把我们解析出来的结果变成一个 dictionary，一个是依据这个 dictionary 依照我们的定义的方式进行替换。

将结果变成dictionary，因为解析的结果是诸如[('?X', '3'), ('?Y', '2')]或者[('?P', ['My', 'dog']), ('?X', ['my', 'cat', 'is', 'very', 'cute'])]

需要判断列表中每个元素的第二项是否为列表，若为列表需要join操作转换为字符串添加，若不是列表(单个字符串)则可以直接添加

In [67]:
def pat_to_dict(patterns):
    return {k: ' '.join(v) if isinstance(v, list) else v for k, v in patterns}

In [68]:
pat_to_dict(pat_match_with_seg("?X greater than ?Y".split(), "3 greater than 2".split()))

{'?X': '3', '?Y': '2'}

In [69]:
pat_to_dict(pat_match_with_seg('?*P is very good and ?*X'.split(), "My dog is very good and my cat is very cute".split()))

{'?P': 'My dog', '?X': 'my cat is very cute'}

接下来是替换，函数输入rule为模板，parsed_rules为上个函数得到的字典。函数递归遍历列表中的每一位，将

In [70]:
def subsitite(rule, parsed_rules):
    #递归边界，已经递归完每一位所以rule为空，返回空列表退出递归
    if not rule: 
        return [] 
    #若rule[0]不存在于parsed_rules，则默认返回rule[0]
    return [parsed_rules.get(rule[0], rule[0])] + subsitite(rule[1:], parsed_rules)

In [77]:
#注意这里任何字符不能与?想连，否则split时会和?为一体即不存在于parsed_rules内
subsitite("first is : ?P second is : ?X".split(), 
          pat_to_dict(pat_match_with_seg('?*P is very good and ?*X'.split(), "My dog is very good and my cat is very cute".split())))

['first', 'is', ':', 'My dog', 'second', 'is', ':', 'my cat is very cute']

现在终于可以完成get_response函数了

In [313]:
import random
def get_response(saying, rules):
    #首先将saying和rules的key逐一匹配，获得正确的patterns(包含变量与对应值的列表)
    for pattern in rules.keys():
        temp=pat_match_with_seg(pattern.split(), saying.split())
        #len(temp[0][1])!=len(saying.split())是防止匹配不上，则将整个saying返回的情况。temp[0][1]是返回variable的值
        #若不加上，"I was ?*X" 将被匹配为"?*X hello ?*Y"的情况(因为?*X在开头会返回整个saying匹配上)
        if temp!='匹配失败'and len(temp[0][1])!=len(saying.split()):  
            patterns=temp
            #提取出该pattern对应的回复模板
            rule=rules[pattern]
            break
            return '输入无法匹配'
    #将patterns变成dictionary
    parsed_rules=pat_to_dict(patterns)
    print(parsed_rules)
    #随后将回复模板中的变量替换。因为有多个回复模板所以随机取一个
    response=subsitite(random.choice(rule).split(),parsed_rules)
    print(' '.join(response))

In [314]:
rules = {
    "I need ?X": ["Image you will get ?X soon","Why do you need ?X ?"], 
    "My ?X told me something": ["Talk about more about your ?X", "How do you think about your ?X ?"],
    "?*X hello ?*Y": ["Hi, how do you do?"],
    "I was ?*X": ["Were you really ?X ?", "I already knew you were ?X ."]
}

In [315]:
get_response('I need iPhone', rules)

{'?X': 'iPhone'}
Image you will get iPhone soon


In [318]:
get_response('I need iPhone', rules)

{'?X': 'iPhone'}
Why do you need iPhone ?


In [152]:
get_response("My mother told me something", rules)

{'?X': 'mother'}
Talk about more about your mother


In [153]:
get_response("My mother told me something", rules)

{'?X': 'mother'}
How do you think about your mother ?


In [168]:
get_response("I was a super hero", rules)

{'?X': 'a super hero'}
I already knew you were a super hero .


In [172]:
get_response("I was a super hero", rules)

{'?X': 'a super hero'}
Were you really a super hero ?


接下来我们将将程序变成能够支持中文输入的模式。其实处理中文的话，只需要将str.split()换成list(jieba.cut(str))就行了。不妨写一个process函数，使其能根据中英文自动切换。

同时对于pattern比如'?\*x一直?\*y'，我们需要切分成['?\*x','一直','?\*y']

'如果你是?y会怎么样呢？',我们需要切分成['如果', '你', '是', '?y', '会', '怎么样', '呢']

In [360]:
import jieba
import copy
def process(str):
    if any('\u4e00' <= i <= '\u9fa5' for i in str):#汉字
            temp=list(jieba.cut(str))
            output=[]
            for m,n in enumerate(temp):
                if n =='?'and temp[m+1]=='*':
                    output.append(temp[m]+temp[m+1]+temp[m+2])
                elif n =='?':
                    output.append(temp[m]+temp[m+1])
                elif '\u4e00' <= n <= '\u9fa5' :
                    output.append(temp[m])
            return output
    return str.split()

In [361]:
process('how do u do')

['how', 'do', 'u', 'do']

In [362]:
process('你好呀')

['你好', '呀']

In [363]:
process('?*x一直?*y')

['?*x', '一直', '?*y']

In [365]:
process('如果你是?y会怎么样呢？')

['如果', '你', '是', '?y', '会', '怎么样', '呢']

改写函数替换split()

In [404]:
def get_response_pro(saying, rules):
    #首先将saying和rules的key逐一匹配，获得正确的patterns(包含变量与对应值的列表)
    for pattern in rules.keys():
        temp=pat_match_with_seg(process(pattern), process(saying))
        #len(temp[0][1])!=len(saying.split())是防止匹配不上，则将整个saying返回的情况。temp[0][1]是返回variable的值
        #若不加上，"I was ?*X" 将被匹配为"?*X hello ?*Y"的情况(因为?*X在开头会返回整个saying匹配上)
        #temp[-1][0][-1]==process( 'saying')[-1][-1] ,前者是提取出的最后一个变量后者是输入的最后一个变量。这样做确保最后一个变量相同
        #不加这条的话，'?*x我是?*y吗'会匹配到'?*x我?*z梦见?*y'，?*z会匹配后面所有字符
        if temp!='匹配失败'and len(temp[0][1])!=len(process(saying)) and temp[-1][0][-1]==process(pattern)[-1][-1]:  
            patterns=temp
            #提取出该pattern对应的回复模板
            rule=rules[pattern]
            print(pattern)
            break
        return '我不太确定我很理解你说的, 能稍微详细解释一下吗?'
    #将patterns变成dictionary
    parsed_rules=pat_to_dict(patterns)
    print(parsed_rules)
    #随后将回复模板中的变量替换。因为有多个回复模板所以随机取一个
    response=subsitite(process(random.choice(rule)),parsed_rules)
    #中文输出汉字间不需要空格，英文单词间需要有空格
    if any('\u4e00' <= i <= '\u9fa5' for i in response):
        print(''.join(response).replace(' ',''))
    else:
        print(' '.join(response))

In [389]:
rule_responses = {
    '?*x hello ?*y': ['How do you do', 'Please state your problem'],
    '?*x I want ?*y': ['what would it mean if you got ?y', 'Why do you want ?y', 'Suppose you got ?y soon'],
    '?*x if ?*y': ['Do you really think its likely that ?y', 'Do you wish that ?y', 'What do you think about ?y', 'Really-- if ?y'],
    '?*x no ?*y': ['why not?', 'You are being a negative', 'Are you saying \'No\' just to be negative?'],
    '?*x I was ?*y': ['Were you really', 'Perhaps I already knew you were ?y', 'Why do you tell me you were ?y now?'],
    '?*x I feel ?*y': ['Do you often feel ?y ?', 'What other feelings do you have?'],
    '?*x你好?*y': ['你好呀', '请告诉我你的问题'],
    '?*x我想?*y': ['你觉得?y有什么意义呢？', '为什么你想?y', '你可以想想你很快就可以?y了'],
    '?*x我想要?*y': ['?x想问你，你觉得?y有什么意义呢?', '为什么你想?y', '?x觉得... 你可以想想你很快就可以有?y了', '你看?x像?y不', '我看你就像?y'],
    '?*x喜欢?*y': ['喜欢?y的哪里？', '?y有什么好的呢？', '你想要?y吗？'],
    '?*x讨厌?*y': ['?y怎么会那么讨厌呢?', '讨厌?y的哪里？', '?y有什么不好呢？', '你不想要?y吗？'],
    '?*xAI?*y': ['你为什么要提AI的事情？', '你为什么觉得AI要解决你的问题？'],
    '?*x机器人?*y': ['你为什么要提机器人的事情？', '你为什么觉得机器人要解决你的问题？'],
    '?*x对不起?*y': ['不用道歉', '你为什么觉得你需要道歉呢?'],
    '?*x我记得?*y': ['你经常会想起这个吗？', '除了?y你还会想起什么吗？', '你为什么和我提起?y'],
    '?*x如果?*y': ['你真的觉得?y会发生吗？', '你希望?y吗?', '真的吗？如果?y的话', '关于?y你怎么想？'],
    '?*x我?*z梦见?*y':['真的吗? --- ?y', '你在醒着的时候，以前想象过?y吗？', '你以前梦见过?y吗'],
    '?*x妈妈?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '她对你影响很大吗？'],
    '?*x爸爸?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '他对你影响很大吗？', '每当你想起你爸爸的时候， 你还会想起其他的吗?'],
    '?*x我愿意?*y': ['我可以帮你?y吗？', '你可以解释一下，为什么想?y'],
    '?*x我很难过，因为?*y': ['我听到你这么说， 也很难过', '?y不应该让你这么难过的'],
    '?*x难过?*y': ['我听到你这么说， 也很难过',
                 '不应该让你这么难过的，你觉得你拥有什么，就会不难过?',
                 '你觉得事情变成什么样，你就不难过了?'],
    '?*x就像?*y': ['你觉得?x和?y有什么相似性？', '?x和?y真的有关系吗？', '怎么说？'],
    '?*x和?*y都?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x和?*y一样?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x我是?*y': ['真的吗？', '?x想告诉你，或许我早就知道你是?y', '你为什么现在才告诉我你是?y'],
    '?*x我是?*y吗': ['如果你是?y会怎么样呢？', '你觉得你是?y吗', '如果你是?y，那一位着什么?'],
    '?*x你是?*y吗':  ['你为什么会对我是不是?y感兴趣?', '那你希望我是?y吗', '你要是喜欢， 我就会是?y'],
    '?*x你是?*y' : ['为什么你觉得我是?y'],
    '?*x因为?*y' : ['?y是真正的原因吗？', '你觉得会有其他原因吗?'],
    '?*x我不能?*y': ['你或许现在就能?*y', '如果你能?*y,会怎样呢？'],
    '?*x我觉得?*y': ['你经常这样感觉吗？', '除了到这个，你还有什么其他的感觉吗？'],
    '?*x我?*y你?*z': ['其实很有可能我们互相?y'],
    '?*x你为什么不?*y': ['你自己为什么不?y', '你觉得我不会?y', '等我心情好了，我就?y'],
    '?*x好的?*y': ['好的', '你是一个很正能量的人'],
    '?*x嗯嗯?*y': ['好的', '你是一个很正能量的人'],
    '?*x不嘛?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x不要?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x有些人?*y': ['具体是哪些人呢?'],
    '?*x有的人?*y': ['具体是哪些人呢?'],
    '?*x某些人?*y': ['具体是哪些人呢?'],
    '?*x每个人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x所有人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x总是?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x一直?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x或许?*y': ['你看起来不太确定'],
    '?*x可能?*y': ['你看起来不太确定'],
    '?*x他们是?*y吗？': ['你觉得他们可能不是?y？'],
    '?*x': ['很有趣', '请继续', '我不太确定我很理解你说的, 能稍微详细解释一下吗?']
}

In [392]:
get_response_pro('高老师我是程序员吗', rule_responses)

?*x我是?*y
{'?x': '高 老师', '?y': '程序员 吗'}
高老师想告诉你或许我早就知道你是程序员吗


In [393]:
get_response_pro('我喜欢编程', rule_responses)

?*x喜欢?*y
{'?x': '我', '?y': '编程'}
编程有什么好的呢


In [395]:
get_response_pro('Sir I want coding', rule_responses)

?*x I want ?*y
{'?x': 'Sir', '?y': 'coding'}
what would it mean if you got coding


In [396]:
get_response_pro('今天我中午梦见编程了', rule_responses)

?*x我?*z梦见?*y
{'?x': '今天', '?z': '中文', '?y': '编程 了'}
你以前梦见过编程了吗


In [397]:
get_response_pro('同学你为什么不做数学题', rule_responses)

?*x你为什么不?*y
{'?x': '同学', '?y': '做 数学题'}
你觉得我不会做数学题


In [405]:
get_response_pro('一加一等于多少', rule_responses)

'我不太确定我很理解你说的, 能稍微详细解释一下吗?'

In [406]:
get_response_pro('1+1=?', rule_responses)

'我不太确定我很理解你说的, 能稍微详细解释一下吗?'

数据驱动在这里的体现就是所有回答都会基于传入的rules。然而人工完成大量的rules编写需要时间精力，如果系统单纯作为闲聊用途的话通过大规模语料集直接训练的语言模型可以节省一定的人力吧。